In [17]:
from scipy.io import loadmat
import pandas as pd
import numpy as np

데이터 로드

In [2]:
df = loadmat('mnist-original.mat')

In [3]:
df

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Sun Mar 30 03:19:02 2014',
 '__version__': '1.0',
 '__globals__': [],
 'mldata_descr_ordering': array([[array(['label'], dtype='<U5'), array(['data'], dtype='<U4')]],
       dtype=object),
 'data': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'label': array([[0., 0., 0., ..., 9., 9., 9.]])}

In [4]:
data = pd.DataFrame(df['data']) #28*28 이미지 데이터를 1행으로 변환
data = data.T
data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Columns: 784 entries, 0 to 783
dtypes: uint8(784)
memory usage: 52.3 MB


In [6]:
label = pd.DataFrame(df['label'])
label = label.T
label.head(5)

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [7]:
label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 1 columns):
0    70000 non-null float64
dtypes: float64(1)
memory usage: 547.0 KB


In [8]:
# 표본을 적절하게 추출함
idx = 0.0
index = 0
idx_lst = [0]
fh = 0
for i in label[0]:
    index += 1
    fh += 1
    if i != idx:
        idx_lst.append(index)
        idx = i
        fh = 0
    elif fh <500:
        idx_lst.append(index)
print(len(idx_lst))
print(idx_lst[-1])

10000
69491


Mnist 데이터 분할 (8:2 비율로 train set, test set split)

In [9]:
from sklearn.model_selection import train_test_split

X = data.iloc[idx_lst]
y = label.iloc[idx_lst]
tr, td, y_tr, y_td = train_test_split(X, y, test_size=0.2, random_state=0)

시간 측정

In [10]:
import time

원본 데이터 & PCA 축소 데이터 & LDA 축소 데이터 비교

In [11]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [40]:
# PCA 축소 데이터
pca = PCA(n_components=0.95) # 누적 분산 비율이 95%가 되도록 차원 선택
tr_pca_reduced = pca.fit_transform(tr)
print('선택한 차원 수 :', pca.n_components_)

선택한 차원 수 : 151


In [41]:
td_pca_reduced = pca.transform(td)

In [43]:
# LDA 축소 데이터
lda = LinearDiscriminantAnalysis(n_components=151)
tr_lda_reduced = lda.fit_transform(tr, y_tr)
td_lda_reduced = lda.transform(td)

C:\Users\john\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\john\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [48]:
tr_lda_reduced.shape

(8000, 9)

In [49]:
td_lda_reduced.shape

(2000, 9)

knn

In [44]:
# 원본 데이터
start = time.time()

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(tr, y_tr)
pred_knn_raw = neigh.predict(td)

print("걸린 시간: {} 초".format(time.time()-start))

C:\Users\john\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


걸린 시간: 36.60614371299744 초


In [46]:
# PCA 축소 데이터
start = time.time()

p_neigh = KNeighborsClassifier(n_neighbors=3)
p_neigh.fit(tr_pca_reduced, y_tr)
pred_knn_pca = p_neigh.predict(td_pca_reduced)

print("걸린 시간: {} 초".format(time.time()-start))

C:\Users\john\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """


걸린 시간: 8.52124834060669 초


In [52]:
tr_lda_reduced.shape, td_lda_reduced.shape

((8000, 9), (2000, 9))

In [58]:
# LDA 축소 데이터
start = time.time()

l_neigh = KNeighborsClassifier(n_neighbors=3)
l_neigh.fit(tr_lda_reduced, y_tr)
pred_knn_lda = l_neigh.predict(td_lda_reduced)

print("걸린 시간: {} 초".format(time.time()-start))

C:\Users\john\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """


걸린 시간: 0.21043944358825684 초


나이브 베이즈

In [79]:
# 원본 데이터
start = time.time()

from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(tr, y_tr)
pred_gnb_raw = gnb.predict(td)

print("걸린 시간: {} 초".format(time.time()-start))

C:\Users\john\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


걸린 시간: 0.6103665828704834 초


In [80]:
# PCA 축소 데이터
start = time.time()

gnb_p = GaussianNB()
gnb_p.fit(tr_pca_reduced, y_tr)
pred_gnb_pca = gnb_p.predict(td_pca_reduced)

print("걸린 시간: {} 초".format(time.time()-start))

걸린 시간: 0.1495974063873291 초


C:\Users\john\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [81]:
# LDA 축소 데이터
start = time.time()

gnb_l = GaussianNB()
gnb_l.fit(tr_lda_reduced, y_tr)
pred_gnb_lda = gnb_l.predict(td_lda_reduced)

print("걸린 시간: {} 초".format(time.time()-start))

걸린 시간: 0.021945476531982422 초


C:\Users\john\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


test accuracy 비교

In [77]:
# knn
from sklearn.metrics import accuracy_score

print("일반 데이터: ", accuracy_score(y_td, pred_knn_raw))
print("PCA 데이터: ", accuracy_score(y_td, pred_knn_pca))
print("LDA 데이터: ", accuracy_score(y_td, pred_knn_lda))

일반 데이터:  0.931
PCA 데이터:  0.936
LDA 데이터:  0.874


In [82]:
# nb

print("일반 데이터: ", accuracy_score(y_td, pred_gnb_raw))
print("PCA 데이터: ", accuracy_score(y_td, pred_gnb_pca))
print("LDA 데이터: ", accuracy_score(y_td, pred_gnb_lda))

일반 데이터:  0.576
PCA 데이터:  0.841
LDA 데이터:  0.857


차원 축소한 데이터에 대해 모델 적용과 연산이 빠르게 일어나고, knn은 차원을 하나도 버리지 않은 데이터를 적용할 경우의 정확도에 비교했을 때 많이 떨어지지 않았고, 나이브베이즈의 경우에는 오히려 더 높은 정확도를 보인다.